# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import api_key

### Import random city data

In [2]:
data = os.path.join(os.getcwd(), '..', 'WeatherPy','output','city_weather.csv')
df = pd.read_csv(data, index_col=0)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,89.01,42,0,11.50,ZA,1616508918
1,Hobart,-42.8794,147.3294,64.00,88,40,6.91,AU,1616509005
2,Castro,-24.7911,-50.0119,77.40,63,37,3.83,BR,1616509056
3,Butaritari,3.0707,172.7902,81.25,79,35,20.58,KI,1616508915
4,Xifeng,42.7372,124.7222,45.01,42,100,2.71,CN,1616509057


### Humidity Heatmap

In [3]:
gmaps.configure(api_key=api_key)
locations = df[['Lat','Lng']]
weights = df['Humidity']
fig = gmaps.figure(center=(0,0),zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame of ideal cities based on following weather criteria
* Max temperature lower than 80 degrees but higher than 70 degrees
* Wind speed less than 10 mph
* 0% cloudiness

In [4]:
ideal_df = df.loc[(df['Cloudiness'] == 0) & (df['Wind Speed'] < 10) & (df['Max Temp'] > 70) & (df['Max Temp'] < 80)]
ideal_df = ideal_df.dropna(how='any')
ideal_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Kruisfontein,-34.0033,24.7314,78.01,57,0,8.01,ZA,1616509063
99,Pisco,-13.7000,-76.2167,71.60,73,0,4.61,PE,1616509081
220,La Libertad,-2.2333,-80.9000,78.80,78,0,4.61,EC,1616509116
238,Cervo,43.6702,-7.4101,71.01,28,0,5.99,ES,1616509121
379,Vallenar,-28.5708,-70.7581,73.47,38,0,3.11,CL,1616509164
407,Tura,25.5198,90.2201,77.40,17,0,2.71,IN,1616509172
417,Lázaro Cárdenas,17.9583,-102.2000,71.08,56,0,3.71,MX,1616509175
551,Champerico,14.3000,-91.9167,75.20,83,0,6.22,GT,1616509218
561,Necochea,-38.5473,-58.7368,73.00,56,0,3.00,AR,1616509221


### Find nearest hotel to each ideal city

In [5]:
hotels = []
for index, row in ideal_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    try:
        res = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=lodging&key={api_key}')
        hotel = res.json()['results'][0]['name']
        hotels.append(hotel)
        print(f"Found hotel near {row['City']}: {hotel}")
    except:
        print(f"No hotels found near {row['City']}.")
        hotels.append('No hotels nearby')

Found hotel near Kruisfontein: Oyster Bay House Rental
Found hotel near Pisco: Hotel San Isidro Oficial
Found hotel near La Libertad: Playa Canela Hotel Boutique
Found hotel near Cervo: Hotel Spa Galatea
Found hotel near Vallenar: Humacao Bed & Breakfast
Found hotel near Tura: Hotel RIKMAN Continental
Found hotel near Lázaro Cárdenas: Hotel Quinta Antigua
Found hotel near Champerico: Hotel y restaurante "El Diamante"
Found hotel near Necochea: dyd -mar Hotel


In [6]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Kruisfontein,-34.0033,24.7314,78.01,57,0,8.01,ZA,1616509063,Oyster Bay House Rental
99,Pisco,-13.7000,-76.2167,71.60,73,0,4.61,PE,1616509081,Hotel San Isidro Oficial
220,La Libertad,-2.2333,-80.9000,78.80,78,0,4.61,EC,1616509116,Playa Canela Hotel Boutique
238,Cervo,43.6702,-7.4101,71.01,28,0,5.99,ES,1616509121,Hotel Spa Galatea
379,Vallenar,-28.5708,-70.7581,73.47,38,0,3.11,CL,1616509164,Humacao Bed & Breakfast
407,Tura,25.5198,90.2201,77.40,17,0,2.71,IN,1616509172,Hotel RIKMAN Continental
417,Lázaro Cárdenas,17.9583,-102.2000,71.08,56,0,3.71,MX,1616509175,Hotel Quinta Antigua
551,Champerico,14.3000,-91.9167,75.20,83,0,6.22,GT,1616509218,"Hotel y restaurante ""El Diamante"""
561,Necochea,-38.5473,-58.7368,73.00,56,0,3.00,AR,1616509221,dyd -mar Hotel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

### Add markers for each hotel to heatmap

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))